In [12]:
import pandas as pd
import sys

sys.path.append("../src")
from token_wrapper import call_tokenizer_csharp
from dafny_utils import extract_dafny_functions

In [3]:
benchmarck_libraries = pd.read_csv("../results/placeholder_dataset/libraries.csv")

In [4]:
benchmarck_libraries.head()

,Index,Original File,Original Method,Original Method Time,Original Method Result,Original Method File,Assertion,Time Difference,New Method File,New Method,New Method Time,New Method Result,New Result File
0,30,/usr/local/home/eric/dafny_repos/libraries/src...,MembersImpl,0.908584,Correct,./results/MembersImpl_0.txt,"assert wr == MembersSpec(obj, members[..0], wr...",NaN,.//results/MembersImpl_fix_30.dfy,MembersImpl,1.000806,Errors,./results/MembersImpl_30.txt
1,31,/usr/local/home/eric/dafny_repos/libraries/src...,MembersImpl,0.908584,Correct,./results/MembersImpl_0.txt,assert members[..i+1][..i] == members[..i];\n,NaN,.//results/MembersImpl_fix_31.dfy,MembersImpl,2.332830,Errors,./results/MembersImpl_31.txt
2,32,/usr/local/home/eric/dafny_repos/libraries/src...,MembersImpl,0.908584,Correct,./results/MembersImpl_0.txt,assert members[..|members|] == members;\n,NaN,.//results/MembersImpl_fix_32.dfy,MembersImpl,1.838043,Errors,./results/MembersImpl_32.txt
3,39,/usr/local/home/eric/dafny_repos/libraries/src...,LemmaIsSingleton,0.476277,Correct,./results/LemmaIsSingleton_0.txt,assert s == {x};\n,NaN,.//results/LemmaIsSingleton_fix_39.dfy,LemmaIsSingleton,0.503212,Errors,./results/LemmaIsSingleton_39.txt
4,42,/usr/local/home/eric/dafny_repos/libraries/src...,LemmaUnionSize,0.553964,Correct,./results/LemmaUnionSize_0.txt,assert xr + yr == xs + ys - {y};\n,NaN,.//results/LemmaUnionSize_fix_42.dfy,LemmaUnionSize,0.595733,Errors,./results/LemmaUnionSize_42.txt


In [18]:
def get_tokens_from_file(method_file, method_name):
    with open(method_file, "r") as file:
        method_file_content = file.read()
    method = extract_dafny_functions(method_file_content, method_name)
    return call_tokenizer_csharp(method), method

In [23]:
df_tokens = pd.DataFrame()

for index, row in benchmarck_libraries.iterrows():
    method_name = row["Original Method"]
    method_file = row["Original File"]
    method_without_assertions_file = f"../{row['New Method File']}"
    tokens_method, method = get_tokens_from_file(method_file, method_name)
    tokens_method_without_assertion, method_without_assertions = get_tokens_from_file(
        method_without_assertions_file, method_name
    )
    new_row = {
        "method_name": method_name,
        "tokens_method": tokens_method,
        "tokens_method_without_assertion": tokens_method_without_assertion,
        "method": method,
        "method_without_assertions": method_without_assertions,
    }
    df_tokens = pd.concat([df_tokens, pd.DataFrame([new_row])], ignore_index=True)
df_tokens.head()

,method_name,tokens_method,tokens_method_without_assertion,method,method_without_assertions
0,MembersImpl,"[[{""Item1"":""67"",""Item2"":""method""},{""Item1"":""1""...","[[{""Item1"":""67"",""Item2"":""method""},{""Item1"":""1""...","method MembersImpl(obj: jobject, writer: Wri...","method MembersImpl(obj: jobject, writer: Wri..."
1,MembersImpl,"[[{""Item1"":""67"",""Item2"":""method""},{""Item1"":""1""...","[[{""Item1"":""67"",""Item2"":""method""},{""Item1"":""1""...","method MembersImpl(obj: jobject, writer: Wri...","method MembersImpl(obj: jobject, writer: Wri..."
2,MembersImpl,"[[{""Item1"":""67"",""Item2"":""method""},{""Item1"":""1""...","[[{""Item1"":""67"",""Item2"":""method""},{""Item1"":""1""...","method MembersImpl(obj: jobject, writer: Wri...","method MembersImpl(obj: jobject, writer: Wri..."
3,LemmaIsSingleton,"[[{""Item1"":""54"",""Item2"":""lemma""},{""Item1"":""1"",...","[[{""Item1"":""54"",""Item2"":""lemma""},{""Item1"":""1"",...",lemma LemmaIsSingleton<T>(s: set<T>)\n en...,lemma LemmaIsSingleton<T>(s: set<T>)\n en...
4,LemmaUnionSize,"[[{""Item1"":""54"",""Item2"":""lemma""},{""Item1"":""1"",...","[[{""Item1"":""54"",""Item2"":""lemma""},{""Item1"":""1"",...","lemma LemmaUnionSize<X>(xs: set<X>, ys: set<...","lemma LemmaUnionSize<X>(xs: set<X>, ys: set<..."


In [24]:
len(df_tokens)

58